In [1]:
import sys
sys.path[0]+='/..'
# from graph_dataset.dataset import GraphDataset
from dataset import GraphDataset

from model.gcn import GNNPolicy
from utilities import *
import torch
import torch_geometric
import os 
import time
from torch.optim.lr_scheduler import CosineAnnealingLR
from readdataUC import UC
import gurobipy as gp
import json


In [2]:
#set folder
uc_model = '3bin'
instance = '60_c11_based_8_std'
model = 'gcn'

model_save_path = f'../model_save/{uc_model}_{instance}_{model}/'
log_save_path =   f"../model_save/{uc_model}_{instance}_{model}_log/"
if not os.path.isdir(model_save_path):
    os.mkdir(model_save_path)
if not os.path.isdir(log_save_path):
    os.mkdir(log_save_path)

log_file = open(f'{log_save_path}train.log', 'wb')

In [3]:
LEARNING_RATE = 1e-4
NB_EPOCHS =2000
BATCH_SIZE = 1
NUM_WORKERS = 2
ETA_MIN=1e-5

In [4]:
train_data = GraphDataset(f'../datasets/{instance}/{uc_model}/train/')
valid_data = GraphDataset(f'../datasets/{instance}/{uc_model}/valid/')
# train_data = GraphDataset(f'../datasets/mixed/train/')
# valid_data = GraphDataset(f'../datasets/mixed/valid/')
train_iter = torch_geometric.loader.DataLoader(
    train_data, batch_size = BATCH_SIZE, shuffle = True,
    num_workers = NUM_WORKERS, pin_memory=True)
valid_iter = torch_geometric.loader.DataLoader(
    valid_data, batch_size = BATCH_SIZE, shuffle = False,
    num_workers = NUM_WORKERS)

In [5]:
# DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
PredictModel = GNNPolicy().to(DEVICE)
# if os.path.exists(model_save_path+'model_best.pth'):
#     state_dict = torch.load(model_save_path+'model_best.pth')
#     PredictModel.load_state_dict(state_dict)
#     print("载入模型成功")

In [6]:
def train(predict, data_loader, optimizer=None):
    """
    This function will process a whole epoch of training or validation, depending on whether an optimizer is provided.
    """
    loss_fn = torch.nn.BCELoss()

    if optimizer:
        predict.train()
    else:
        predict.eval()
    mean_loss = 0
    n_samples_processed = 0
    with torch.set_grad_enabled(optimizer is not None):
        for step, batch in enumerate(data_loader):
            batch = batch.to(DEVICE)
            weighted_sol = batch.opt_sol
            #predict the binary distribution, BD
            BD = predict(
                batch.constraint_features,
                batch.edge_index,
                batch.edge_attr,
                batch.variable_features,
            )
            # compute loss
            # print(batch.constraint_features.shape,batch.variable_features.shape)
            presol = BD[batch.b_vars==1]
            presol = presol.sigmoid()
            loss = loss_fn(presol, weighted_sol)
            
            

            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            mean_loss += loss.item()
            n_samples_processed += 1
    mean_loss /= n_samples_processed

    return mean_loss


In [7]:
def ps_accelerate(sol,file):
    
    with gp.Env(empty=True) as env:
        env.setParam('OutputFlag', 0)
        env.setParam('MIPGap', 1e-3)
        env.start()
        m = gp.read(file,env)
        f = m.getObjective()
        k=1
        
        sol = he_fs(m,sol,k,instance)
        sol = remove_initial(sol,m)
        day = int(file.split('/')[-1].split('_')[0])
        sol = restore_on_off(sol,instance,day)
        m = fix_variable(m,sol)

        m.Params.TimeLimit = 10
        m.optimize()
        if m.Status==2 or m.Status ==9:
            return f.getValue()
        else:
            return -1

In [8]:
def valid(predict):
    lp_path=f'../datasets/{instance}/{uc_model}/lp/valid/'
    bin3_path=f'../datasets/{instance}/3bin/lp/valid/'
    with open(f'../datasets/{instance}/{uc_model}/lp/valid/obj.json', 'r') as f:
        objs = json.load(f)
    inf_cnt=0
    opt_loss=[]
    for file in os.listdir(lp_path):
        if file.endswith('.lp'):
            lp_file = os.path.join(lp_path, file)
            bin3_file = os.path.join(bin3_path, file)
            with gp.Env(empty=True) as env:
                env.setParam('OutputFlag', 0)
                env.setParam('MIPGap', 1e-3)
                env.start()
                m = gp.read(lp_file,env)
            A_indices, A_values, v_nodes, c_nodes, b_vars=get_fea(m)
            sol = predict(c_nodes.to(DEVICE), A_indices.to(DEVICE), A_values.to(DEVICE), v_nodes.to(DEVICE)).sigmoid()  
            sol = sol[b_vars==1]


            obj = ps_accelerate(sol,bin3_file)
            if obj == -1:
                inf_cnt+=1
            else:
                opt_loss.append(obj/objs[file.split('.')[0]])
    return sum(opt_loss)/len(opt_loss)-1,inf_cnt/(len(opt_loss)+inf_cnt)

In [9]:
optimizer = torch.optim.AdamW(PredictModel.parameters(), lr=LEARNING_RATE,weight_decay=1e-6)
scheduler = CosineAnnealingLR(optimizer, T_max=NB_EPOCHS/10, eta_min=ETA_MIN)
best_val_loss = 0.2
best_acc=1

In [10]:
cur_epoch = 0
for epoch in range(NB_EPOCHS):
    begin=time.time()
    cur_epoch+=1
    train_loss = train(PredictModel, train_iter, optimizer)
    print(f"Epoch {epoch} Train loss: {train_loss:0.6f}",end='\t')
    valid_loss = train(PredictModel, valid_iter, None)
    print(f"Epoch {epoch} Valid loss: {valid_loss:0.6f}")
    acc,infs=valid(PredictModel)
    print(acc)
    if acc<best_acc:
        cur_epoch=0
        best_acc = acc
        print(best_acc)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best.pth')
    torch.save(PredictModel.state_dict(), model_save_path+'model_last.pth')
    #100个epochloss不下降就停止
    if cur_epoch>100:
        break
    date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
    st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}    TIME:{time.time()-begin}    DONE_TIME:{date_str}\n'
    log_file.write(st.encode())
    log_file.flush()
    scheduler.step()
print('done')

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/peilun/miniconda3/envs/gat/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/peilun/miniconda3/envs/gat/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/peilun/miniconda3/envs/gat/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/home/peilun/miniconda3/envs/gat/lib/python3.11/site-packages/torch_geometric/data/dataset.py", line 258, in __getitem__
    data = self.get(self.indices()[idx])
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  F